In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json

# Initialize Spark session
spark = SparkSession.builder.appName("ImportFourFormats").getOrCreate()

# 1. Create and Import CSV
csv_data = """name,age
Harsh,22
Hari,18"""
dbutils.fs.put("/FileStore/sample.csv", csv_data, overwrite=True)
csv_df = spark.read.option("header", "true").csv("/FileStore/sample.csv")
csv_df.write.mode("overwrite").format("delta").saveAsTable("csv_table")

# 2. Create and Import JSON
json_data = [{"name": "Puneeth", "age": 28}, {"name": "Raju", "age": 35}]
dbutils.fs.put("/FileStore/sample.json", json.dumps(json_data), overwrite=True)
json_df = spark.read.json("/FileStore/sample.json")
json_df.write.mode("overwrite").format("delta").saveAsTable("json_table")

# 3. Create and Import Parquet
parquet_df = spark.createDataFrame([("Prashanth", 22), ("Darshan", 40)], ["name", "age"])
parquet_df.write.mode("overwrite").parquet("/FileStore/sample.parquet")
parquet_read_df = spark.read.parquet("/FileStore/sample.parquet")
parquet_read_df.write.mode("overwrite").format("delta").saveAsTable("parquet_table")

# 4. Create and Import Avro
# Note: Avro requires the spark-avro package, included in Databricks
avro_df = spark.createDataFrame([("Rahul", 27), ("Gagan", 33)], ["name", "age"])
avro_df.write.mode("overwrite").format("avro").save("/FileStore/sample.avro")
avro_read_df = spark.read.format("avro").load("/FileStore/sample.avro")
avro_read_df.write.mode("overwrite").format("delta").saveAsTable("avro_table")

# Display tables (for verification)
display(spark.sql("SHOW TABLES"))

Wrote 25 bytes.
Wrote 61 bytes.


database,tableName,isTemporary
default,avro_table,false
default,csv_ingested_table,false
default,csv_table,false
default,existing_table,false
default,json_table,false
default,new_delta_table,false
default,parquet_table,false


In [0]:
%sql
SELECT * FROM avro_table;

name,age
Rahul,27
Gagan,33


In [0]:
%sql
SELECT * FROM parquet_table;

name,age
Prashanth,22
Darshan,40


In [0]:
%sql
SELECT * FROM json_table;

age,name
28,Puneeth
35,Raju


In [0]:
%sql
SELECT * FROM csv_table;

name,age
Harsh,22
Hari,18
